# ASSIGNMENT 2

# Part 3 (Build and Train an RNN on MNIST)

After training our network, we had a Loss = 0.069390 and Training Accuracy = 0.98000. Also, the Testing Accuracy = 0.9804. WHich is a very good result.

However, after we changed line 35 in our code to rnn_cell.BasicRNNCell(nHidden) in order to use LSTM and GRU instead of RNN. We had a Loss = 0.051129 and Training Accuracy = 0.99000 with Testing Accuracy = 0.9769. Also a very good outcome. However, the formal is better.

Comparing these two results to training using convnet in assignment 1. In assignment 1, We had a Training Accuracy = 0.98 with Testing Accuracy = 0.9883. 

All three networks gives great outcome, however, training using convnet has the highest Testing Accuracy but it takes the highest c time.

In [2]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

learningRate =.0001
trainingIters = 50
batchSize = 100
displayStep = 10

nInput = 28 #we want the input to take the 28 pixels
nSteps = 28 #every 28
nHidden = 128 #number of neurons for the RNN
nClasses = 10 #this is MNIST so you know

x = tf.placeholder('float', [None, nSteps, nInput])
y = tf.placeholder('float', [None, nClasses])

weights = {
    'out': tf.Variable(tf.random_normal([nHidden, nClasses]))
}

biases = {
    'out': tf.Variable(tf.random_normal([nClasses]))
}

def RNN(x, weights, biases):
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, nInput])
    x = tf.split(x, nSteps, 0) #configuring so you can get it as needed for the 28 pixels

    lstmCell = rnn_cell.BasicLSTMCell(nHidden,state_is_tuple=True)#find which lstm to use in the documentation

    outputs, states = rnn.static_rnn(lstmCell, x, dtype=tf.float32)#for the rnn where to get the output and hidden state

    return tf.matmul(outputs[-1], weights['out'])+ biases['out']

pred = RNN(x, weights, biases)

#optimization
#create the cost, optimization, evaluation, and accuracy
#for the cost softmax_cross_entropy_with_logits seems really good
cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learningRate).minimize(cost)

correctPred =tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, 'float'))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # while step* batchSize < trainingIters:
    for step in range(trainingIters):

        for _ in range(int(mnist.train.num_examples / batchSize)):
            batchX, batchY = mnist.train.next_batch(batchSize)#mnist has a way to get the next batch
            batchX = batchX.reshape((batchSize, nSteps, nInput))

            sess.run(optimizer, feed_dict={x: batchX, y:batchY})

        if step % displayStep == 0:
            acc = sess.run(accuracy, feed_dict={x: batchX, y:batchY})
            loss = sess.run(cost, feed_dict={x: batchX, y:batchY})
            print("Iter " + str(step) + ", Minibatch Loss= " +
                  "{:.6f}".format(loss) + ", Training Accuracy= " +
                  "{:.5f}".format(acc))

    print('Optimization finished')

    testData = mnist.test.images.reshape((-1, nSteps, nInput))
    testLabel = mnist.test.labels
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: testData, y:testLabel}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


W1105 06:51:23.760756  3676 deprecation.py:323] From <ipython-input-2-cf8d60855ef7>:35: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W1105 06:51:23.761754  3676 deprecation.py:323] From <ipython-input-2-cf8d60855ef7>:37: static_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
W1105 06:51:23.799687  3676 deprecation.py:506] From C:\Users\oyeoy\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passi

Iter 0, Minibatch Loss= 0.490804, Training Accuracy= 0.86000
Iter 10, Minibatch Loss= 0.046969, Training Accuracy= 0.98000
Iter 20, Minibatch Loss= 0.043729, Training Accuracy= 0.97000
Iter 30, Minibatch Loss= 0.063595, Training Accuracy= 0.98000
Iter 40, Minibatch Loss= 0.069390, Training Accuracy= 0.98000
Optimization finished
Testing Accuracy: 0.9804
